In [1]:
import requests
url = "https://ressources.data.sncf.com/api/records/1.0/search/"

In [2]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import *

engine = create_engine('sqlite:///db.sqlite')
Base = declarative_base()

Session = sessionmaker(bind=engine)
session = Session()

class Record(Base):
    __tablename__ = "Record"

    recordid = Column(String, primary_key=True)
    gare_code_uic = Column(String, nullable=True)
    type = Column(String, nullable=True)
    nature = Column(String, nullable=True)
    recordtype = Column(String, nullable=True)
    date = Column(String, nullable=True)

class Gare(Base):
    __tablename__ = "Gare"

    uic_code = Column(String, primary_key=True)
    departement_libellemin = Column(String, nullable=True)
    gare_regionsncf_libelle = Column(String, nullable=True)
    gare_alias_libelle_noncontraint = Column(String, nullable=True)
    adresse_cp = Column(String, nullable=True)
    code_gare = Column(String, nullable=True)
    lat = Column(REAL, nullable=True)
    long = Column(REAL, nullable=True)


Base.metadata.create_all(engine)


In [3]:
# # APPEL API POUR TOUS LES RECORDS ENTRE 2016 ET 2021

# for a in [2016, 2017, 2018, 2019, 2020, 2021]:
#     for m in ["01","02","03","04","05","06","07","08","09","10","11","12"]:
#         response = requests.get(f'{url}?dataset=objets-trouves-restitution&rows=-1&refine.date={str(a)}%2F{m}').json()
#         try:
#             for i in response['records']:
#                 try:
#                     gare_code_uic = i['fields']['gc_obo_gare_origine_r_code_uic_c']
#                 except:
#                     gare_code_uic = None
                    
#                 session.add(Record(recordid = i['recordid'],gare_code_uic = gare_code_uic,type = i['fields']['gc_obo_type_c'],nature = i['fields']['gc_obo_nature_c'],recordtype = i['fields']['gc_obo_nom_recordtype_sc_c'],date = i['fields']['date']))
#             session.commit()
#         except:
#             pass


In [4]:
# # APPEL API POUR TOUTES LES GARES

# for g in requests.get(f'{url}?dataset=referentiel-gares-voyageurs&q=&facet=departement_libellemin&facet=segmentdrg_libelle&facet=gare_agencegc_libelle&facet=gare_regionsncf_libelle&facet=gare_ug_libelle&rows=-1').json()['records']:
    
#     try:
#         gare_regionsncf_libelle = g['fields']['gare_regionsncf_libelle']
#     except:
#         gare_regionsncf_libelle = None
    
#     try:
#         departement_libellemin = g['fields']['departement_libellemin']
#     except:
#         departement_libellemin = None

#     try:
#         gare_alias_libelle_noncontraint = g['fields']['gare_alias_libelle_noncontraint']
#     except:
#         gare_alias_libelle_noncontraint = None

#     try:
#         adresse_cp = g['fields']['adresse_cp']
#     except:
#         adresse_cp = None

#     try:
#         code_gare = g['fields']['code_gare']
#     except:
#         code_gare = None

#     try:
#         lat = g['geometry']['coordinates'][1]
#         long = g['geometry']['coordinates'][0]
#     except:
#         lat = None
#         long = None

#     try:
#         code_gare = g['fields']['code_gare']
#     except:
#         code_gare = None

#     try:
#         session.add(
#         Gare(
#             uic_code = g['fields']['uic_code'],
#             departement_libellemin = departement_libellemin,
#             gare_regionsncf_libelle = gare_regionsncf_libelle,
#             gare_alias_libelle_noncontraint = gare_alias_libelle_noncontraint,
#             adresse_cp = adresse_cp,
#             code_gare = code_gare,
#             lat = lat,
#             long = long
#             )
#         )
#         session.commit()
#     except:
#         pass
    

In [5]:
import pandas as pd

df_gare = pd.read_sql_table('Gare', 'sqlite:///db.sqlite')
df_record = pd.read_sql_table('Record', 'sqlite:///db.sqlite')

In [7]:
# On regarde les columns
df_gare.columns

Index(['uic_code', 'departement_libellemin', 'gare_regionsncf_libelle',
       'gare_alias_libelle_noncontraint', 'adresse_cp', 'code_gare', 'lat',
       'long'],
      dtype='object')

In [8]:
regions = {
    "Auvergne-Rhône-Alpes": ["01", "03", "07", "15", "26", "38", "42", "43", "63", "69", "73", "74"],
    "Bourgogne-Franche-Comté": ["21", "25", "39", "58", "70", "71", "89", "90"],
    "Bretagne": ["22", "29", "35", "56"],
    "Centre-Val de Loire": ["18", "28", "36", "37", "41", "45"],
    "Corse": ["2A", "2B"],
    "Grand Est": ["08", "10", "51", "52", "54", "55", "57", "67", "68", "88"],
    "Hauts-de-France": ["02", "59", "60", "62", "80"],
    "Île-de-France": ["75", "77", "78", "91", "92", "93", "94", "95"],
    "Normandie": ["14", "27", "50", "61", "76"],
    "Nouvelle-Aquitaine": ["16", "17", "19", "23", "24", "33", "40", "47", "64", "79", "86", "87"],
    "Occitanie": ["09", "11", "12", "30", "31", "32", "34", "46", "48", "65", "66", "81", "82"],
    "Pays de la Loire": ["44", "49", "53", "72", "85"],
    "Provence-Alpes-Côte d'Azur": ["04", "05", "06", "13", "83", "84"],
    "Guadeloupe": ["971"],
    "Martinique": ["972"],
    "Guyane": ["973"],
    "La Réunion": ["974"],
    "Mayotte": ["976"]
}

In [9]:
# On change le nom de la région car c'est les anciennnes régions
def search_region(departement: str):
    for cle, valeur in regions.items():
        if departement in valeur:
            return str(cle)

df_gare['gare_regionsncf_libelle'] = df_gare['adresse_cp'].apply(lambda x: search_region(str(x)[:2]))

In [10]:
# On JOIN les deux dataframe et on retire la colonne "uic_code"
merge = pd.merge(df_record, df_gare, how="left", left_on="gare_code_uic", right_on="uic_code").drop("uic_code", axis=1)
merge.to_csv("sncf.csv", index=False)